In [ ]:
#Dash Setup
from jupyter_dash import JupyterDash
from dash import Dash, html, dcc, dash_table
from dash.dependencies import Input, Output, State
import dash
import pandas as pd
import plotly.express as px

#mmongoDB CRUD module
from AnimalShelter import AnimalShelter

#credentials
shelter = AnimalShelter(
    user='aacuser',
    passwd='Fantastic',
    host='nv-desktop-services.apporto.com',
    port=32063,
    database='aac',
    collection='animals')
# Retrieve all records from the 'animals' collection
df = pd.DataFrame(shelter.read({}))
# Clean up column names by removing any empty or unnamed columns
df = df.loc[:, df.columns.str.strip() !='']

In [ ]:
app = JupyterDash(__name__)
#rename columns
column_mapping = {
    "age_upon_outcome": "Age",
    "animal_id": "Animal ID",
    "animal_type": "Type",
    "breed": "Breed",
    "color": "Color",
    "date_of_birth": "Birth Date",
    "datetime": "Outcome Date",
    "name": "Name",
    "outcome_subtype": "Subtype",
    "outcome_type": "Outcome",
    "sex_upon_outcome": "Sex",
    "location_lat": "Latitude",
    "location_long": "Longitude",
    "age_upon_outcome_in_weeks": "Age(weeks)"
}

app.layout = html.Div([
    
    #header
    html.Div([
        html.A([
            html.Img(src='assets/Grazioso Salvare Logo.png', style={'height': '80px'})
        ], href='https://www.snhu.edu'),
        
        html.H6("Dashboard by George Yockachonis", style={'marginLeft': '20px'})
    ], style={"display": "flex", "alignItems": "center", "gap": "20px"}),
    
    html.Hr(),
    
    html.Div([
        html.H6("Filter by Rescue Type", style={'marginRight': '20px'}),
    #interactive buttons
        dcc.RadioItems(
            id='rescue-type-selector',
            options=[
                {'label' : 'Water Rescue', 'value': 'Water'},
                {'label' : 'Mountain or Wilderness Rescue', 'value': 'Mountain'},
                {'label' : 'Disaster or Individual Tracking', 'value': 'Disaster'},
                {'label' : 'Reset', 'value': 'Reset'},
            ],
            value='Reset',
            labelStyle={'display':'inline-block', 'marginRight': '20px'}
        ),
    ], style={'display': 'flex', 'alignItems': 'center', 'padding': '10px'}),
    
    html.Hr(),
    
    dcc.Store(id='stored-selected-rows', data=[]),
    #Data Table
    dash_table.DataTable(
        id='animal-table',
        columns=[{"name": column_mapping.get(col, col), "id": col}
                 for col in df.columns if col not in ["_id", "monthyear"]],
        data=df.drop(columns=["_id", "monthyear"]).to_dict('records'),
        page_size=5,
        sort_action='native',
        filter_action='native',
        row_selectable='single',
        selected_rows=[],
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left'},
    ),
    
    html.Br(),
    html.Div([
    #map chart
        dcc.Graph(id='location-chart', style={'width': '50%', 'display': 'inline-block'}),
    
    #outcome chart
        dcc.Graph(id='outcome-chart', style={'width': '50%', 'display': 'inline-block'})
    ])
])


In [ ]:
def get_filtered_data(rescue_type):
    if rescue_type == "Water":
        return shelter.read({
            "animal_type": "Dog",
            "breed": {"$regex": "Labrador Retriever|Newfoundland|Chesapeake Bay Retriever", "$options": "i"},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        })
    elif rescue_type == "Mountain":
          return shelter.read({
            "animal_type": "Dog",
            "breed": {"$regex": "German Shepherd|Alaskan Malamute|Old English Sheepdog|Siberian Husky|Rottweiler", "$options": "i"},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        })
    elif rescue_type == "Disaster":
          return shelter.read({
            "animal_type": "Dog",
            "breed": {"$regex": "Doberman Pinscher|German Shepherd|Golden Retriever|Bloodhound|Rottweiler", "$options": "i"},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        })
    else:
        return shelter.read({})

In [ ]:
# This callback updates the table, map, pie chart, and selected row tracking
# based on either user selection (clicking a row) or changing the rescue filter.

@app.callback(
    Output('animal-table', 'data'),               # Update the table data
    Output('location-chart', 'figure'),           # Update the map chart
    Output('outcome-chart', 'figure'),            # Update the pie chart
    Output('animal-table', 'selected_rows'),      # Dynamically update selected row (for deselection)
    Output('stored-selected-rows', 'data'),       # Save last selected row to compare on future clicks
    Input('rescue-type-selector', 'value'),       # User changes filter
    Input('animal-table', 'selected_rows'),       # User clicks a row
    State('stored-selected-rows', 'data')         # Previously selected row (used for toggle logic)
)
def update_dashboard(rescue_type, selected_rows, previous_selected_rows):
    ctx = dash.callback_context

    # Initialize lists safely (handle None)
    selected_rows = selected_rows or []
    previous_selected_rows = previous_selected_rows or []

    return_selected_rows = selected_rows
    updated_stored_rows = selected_rows

    # Identify what triggered the callback (radio button or table row click)
    trigger_id = ctx.triggered[0]['prop_id'].split('.')[0] if ctx.triggered else None

    # === Deselect logic ===
    # If the filter was changed, clear any selected row
    if trigger_id == 'rescue-type-selector':
        selected_rows = []
        return_selected_rows = []
        updated_stored_rows = []

    # If user clicked the same row again, deselect it
    elif (
        trigger_id == 'animal-table' and
        selected_rows == previous_selected_rows and
        selected_rows != []
    ):
        selected_rows = []
        return_selected_rows = []
        updated_stored_rows = []

    # === Retrieve and clean the filtered data ===
    records = get_filtered_data(rescue_type)
    for r in records:
        r.pop('_id', None)  # Remove MongoDB _id for chart compatibility

    df = pd.DataFrame(records)

    # Rename unnamed index column if it exists
    if 'Unnamed: 0' in df.columns:
        df.rename(columns={"Unnamed: 0": "rec_num"}, inplace=True)

    # Clean and sort record numbers
    if 'rec_num' in df.columns:
        df['rec_num'] = pd.to_numeric(df['rec_num'], errors='coerce')
        df = df.dropna(subset=['rec_num'])
        df = df.sort_values(by='rec_num')

    # === Handle empty result ===
    if df.empty:
        empty_fig = {
            "layout": {
                "title": f"No matching results found for {rescue_type}",
                "xaxis": {"visible": False},
                "yaxis": {"visible": False}
            }
        }
        return [], empty_fig, empty_fig, return_selected_rows, updated_stored_rows

    # === Map Chart ===
    if 'location_lat' in df.columns and 'location_long' in df.columns:
        location_df = df.dropna(subset=['location_lat', 'location_long'])

        map_fig = px.scatter_mapbox(
            location_df,
            lat="location_lat",
            lon="location_long",
            hover_name="breed",
            zoom=10
        )

        # Highlight selected row with a red marker
        if selected_rows and len(selected_rows) > 0:
            selected_index = selected_rows[0]
            if selected_index < len(df):
                selected_row = df.iloc[selected_index]
                if pd.notna(selected_row['location_lat']) and pd.notna(selected_row['location_long']):
                    highlight_df = pd.DataFrame([selected_row])
                    highlight_trace = px.scatter_mapbox(
                        highlight_df,
                        lat="location_lat",
                        lon="location_long",
                        hover_name="breed"
                    ).data[0]

                    # Red marker styling
                    highlight_trace.marker.color = "red"
                    highlight_trace.marker.size = 12
                    map_fig.add_trace(highlight_trace)

                    # Center map on selected dog
                    map_fig.update_layout(
                        mapbox_center={
                            "lat": selected_row['location_lat'],
                            "lon": selected_row['location_long']
                        },
                        mapbox_zoom=12
                    )

        map_fig.update_layout(mapbox_style="open-street-map")
    else:
        # Fallback map if no coordinates are found
        map_fig = {
            "layout": {
                "title": "No location data available",
                "xaxis": {"visible": False},
                "yaxis": {"visible": False}
            }
        }

    # === Pie Chart ===
    pie_fig = px.pie(df, names='outcome_type', title='Outcome Types')

    # Final return: updated data + visuals + selection state
    return df.to_dict('records'), map_fig, pie_fig, return_selected_rows, updated_stored_rows


# Run the Dash server
app.run_server(debug=True)

Dash app running on http://127.0.0.1:28799/
